In [14]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/2"

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

C:\Users\bekasker\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(preprocessor_url)
encoder_inputs = preprocessor(text_input)

In [17]:
encoder = hub.KerasLayer(encoder_url, trainable=True)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [18]:
layer = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [20]:
from tensorflow import keras
from tensorflow.keras import callbacks, metrics 

In [23]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-4,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

reduce_learning_rate = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-7, 
    verbose=1
)

In [24]:
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping, reduce_learning_rate]
)

Epoch 1/10
656/656 [==============================] - 292s 428ms/step - loss: 0.5099 - accuracy: 0.7416 - precision: 0.7387 - recall: 0.7478 - val_loss: 0.3478 - val_accuracy: 0.8532 - val_precision: 0.8464 - val_recall: 0.8629 - lr: 1.0000e-05
Epoch 2/10
656/656 [==============================] - 287s 438ms/step - loss: 0.3567 - accuracy: 0.8437 - precision: 0.8366 - recall: 0.8543 - val_loss: 0.2940 - val_accuracy: 0.8790 - val_precision: 0.8717 - val_recall: 0.8887 - lr: 1.0000e-05
Epoch 3/10
656/656 [==============================] - 284s 434ms/step - loss: 0.3095 - accuracy: 0.8669 - precision: 0.8598 - recall: 0.8767 - val_loss: 0.2779 - val_accuracy: 0.8828 - val_precision: 0.9163 - val_recall: 0.8425 - lr: 1.0000e-05
Epoch 4/10
656/656 [==============================] - 287s 438ms/step - loss: 0.2734 - accuracy: 0.8861 - precision: 0.8802 - recall: 0.8940 - val_loss: 0.2445 - val_accuracy: 0.8988 - val_precision: 0.8892 - val_recall: 0.9110 - lr: 1.0000e-05
Epoch 5/10
656/656 [